In [2]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)


In [3]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()):
    if "anuja" in os.environ.get('USER'):
        DATA_DIR = 'data/'
    elif 'ubuntu' in os.environ.get('USER'):
        DATA_DIR = '/home/ubuntu/Martyna/repo/AI4Health/DATAfoof/'
    

## EEG data and foof:

In [4]:

## data loading
data = mat73.loadmat(DATA_DIR+'x_source_new.mat')  
foof = pd.read_csv(DATA_DIR+"source_intercept_slope.csv")
foof


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,127,128,129,130,131,132,133,134,135,IDs
0,0,0.160123,1.362617,-0.002037,1.427172,0.810964,1.391837,0.745124,1.378194,0.527491,...,1.519565,1.004265,1.450747,0.956578,1.456952,0.611944,1.503932,0.282555,1.441946,NDARAA075AMK
1,1,0.988915,1.330636,1.151667,1.546990,1.479442,1.608810,1.342563,1.559859,0.460923,...,1.461904,1.449481,1.480738,0.889122,1.282076,0.950479,1.138697,1.372408,1.505823,NDARAA112DMH
2,2,1.455004,1.515717,0.988018,1.367703,2.029333,1.763131,2.086803,1.763987,1.320758,...,1.502295,1.596192,1.448922,1.180667,1.199036,1.658822,1.619147,1.182087,1.395998,NDARAA117NEJ
3,3,0.796931,1.547997,0.016022,0.956344,0.638462,1.222323,0.698921,1.395078,-0.130122,...,0.998001,0.079507,1.285272,0.471817,1.175649,0.617125,1.418717,0.401675,1.222953,NDARAA947ZG5
4,4,-0.627481,1.123518,-0.255195,1.273727,0.197328,1.344738,0.282533,1.387126,-0.808075,...,1.363285,0.049249,1.207528,-0.118998,1.296786,-0.253839,1.176320,-0.033039,1.318181,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,2037,-0.561398,1.060676,-0.680878,1.241504,0.177737,1.110332,0.141657,1.116719,-0.347975,...,1.153422,0.182297,1.016177,0.177069,1.042335,-0.124226,1.165734,-0.418218,1.085482,NDARZN277NR6
2038,2038,1.178082,1.344137,1.127791,1.414553,1.802019,1.642233,1.680568,1.593067,0.891970,...,1.576864,2.119400,1.755767,1.780714,1.576273,1.551271,1.384801,1.587596,1.466845,NDARZN578YDP
2039,2039,-0.394953,1.348060,-0.918028,1.088161,-0.134448,1.122479,-0.268877,1.111450,-0.288241,...,1.099186,-0.022279,1.275160,-0.316072,1.201135,-0.249886,1.300831,-0.656477,1.232983,NDARZN610GTY
2040,2040,-0.347506,0.870524,0.144745,1.257984,0.636052,0.952351,0.745472,0.988167,-0.353721,...,1.278362,0.544927,0.883756,0.788643,1.038478,-0.372208,0.747925,0.414824,1.251458,NDARZN677EYE


In [5]:
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df = np.array(df)
df = df.reshape(data['x'].shape)
df.shape

df = pd.DataFrame(df.reshape((df.shape[0], -1)))
df['IDs'] = foof['IDs']
df



,0,1,2,3,4,5,6,7,8,9,...,5363,5364,5365,5366,5367,5368,5369,5370,5371,IDs
0,2.852107e-12,6.843959e-11,1.334486e-09,2.114389e-08,2.722198e-07,2.847860e-06,2.420928e-05,1.672280e-04,9.386428e-04,4.281104e-03,...,4.083400e-13,1.199041e-14,2.220446e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA075AMK
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.422859e-02,1.792809e-01,2.261597e-01,1.471350e-01,4.932961e-02,8.522938e-03,7.588586e-04,3.481949e-05,8.233298e-07,NDARAA112DMH
2,1.873727e-04,3.700203e-02,1.338341e-01,8.866054e-03,1.075761e-05,2.390714e-10,1.143530e-13,4.469758e-12,1.420740e-10,4.289156e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA117NEJ
3,4.679133e-03,2.858350e-02,1.446466e-01,3.296430e-01,3.146601e-01,1.356668e-01,1.148714e-01,3.878903e-01,4.688989e-01,1.760009e-01,...,4.721801e-02,2.446504e-02,6.717445e-02,1.737527e-01,2.703009e-01,2.480023e-01,1.340995e-01,4.273146e-02,8.024469e-03,NDARAA947ZG5
4,1.426186e-01,1.919957e-01,2.337057e-01,2.572226e-01,2.559831e-01,2.303433e-01,1.874140e-01,1.378767e-01,9.171530e-02,5.516572e-02,...,5.047329e-02,4.878144e-02,5.355981e-02,1.013861e-01,5.092052e-02,3.967086e-02,3.699821e-02,3.428801e-02,3.155638e-02,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,1.641564e-04,5.061825e-03,4.748293e-02,1.355030e-01,1.176366e-01,3.107015e-02,2.513731e-03,1.849108e-04,7.014072e-04,3.183943e-03,...,3.161835e-08,1.106568e-08,3.738061e-09,1.218730e-09,3.834910e-10,1.164628e-10,3.413536e-11,9.656276e-12,2.636114e-12,NDARZN277NR6
2038,1.273725e-07,2.854585e-06,4.301847e-05,4.342317e-04,2.934193e-03,1.327155e-02,4.019215e-02,8.169716e-02,1.133810e-01,1.182013e-01,...,6.180279e-12,1.819545e-12,5.200285e-13,1.443290e-13,3.885781e-14,1.021405e-14,2.553513e-15,6.661338e-16,1.110223e-16,NDARZN578YDP
2039,0.000000e+00,7.771561e-16,3.630429e-14,1.391887e-12,4.209189e-11,1.004152e-09,1.889601e-08,2.804845e-07,3.284092e-06,3.033118e-05,...,1.329739e-01,1.350156e-01,1.346921e-01,1.320204e-01,1.271395e-01,1.202986e-01,1.118360e-01,1.021511e-01,9.167378e-02,NDARZN610GTY
2040,1.363800e-08,1.521235e-07,1.428039e-06,1.127810e-05,7.492867e-05,4.187613e-04,1.968741e-03,7.785975e-03,2.590237e-02,7.248831e-02,...,5.546025e-07,2.166196e-07,8.182735e-08,2.989401e-08,1.056221e-08,3.609193e-09,1.192753e-09,3.812199e-10,1.178382e-10,NDARZN677EYE


## Behavioral Data:


In [6]:
behaviour_data = pd.read_csv(DATA_DIR+'AllData.csv')
behaviour_data = behaviour_data.rename(columns = {'EID': 'IDs'}, inplace = False)
behaviour_data.columns

Index(['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_SR_GD', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum'],
      dtype='object')

In [7]:
beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
add_features = beh[['Sex', 'Age','IDs']]
add_features.shape

(3076, 3)

In [8]:
behaviour_data = behaviour_data[['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum','WISC_VCI_Sum']]
behaviour_data.shape

(2579, 8)

In [9]:
# def missing_values_table(df):
#     mis_val = df.isnull().sum()
#     mis_val_percent = 100 * df.isnull().sum() / len(df)
#     mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
#     mis_val_table_ren_columns = mis_val_table.rename(
#     columns = {0 : 'Missing Values', 1 : '% of Total Values'})
#     mis_val_table_ren_columns = mis_val_table_ren_columns[
#         mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
#     '% of Total Values', ascending=False).round(1)
#     print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
#         "There are " + str(mis_val_table_ren_columns.shape[0]) +
#             " columns that have missing values.")
#     return mis_val_table_ren_columns

# missing_values_table(behaviour_data)

## Merge EEG and foof data together:

In [10]:
data = pd.merge(df, foof, on='IDs', how='inner')
data.shape

(2042, 5510)

## and then with the behaviorals

In [11]:
df = pd.merge(data, behaviour_data, on='IDs', how='inner')
df.shape

(1465, 5517)

In [12]:
 #add age and gender
 df = pd.merge(df, add_features, on='IDs', how='inner')
 df.shape

(1443, 5519)

## features and labels preparation:

In [13]:
# #removing NaNs
df = df.dropna()
df.shape

(1072, 5519)

In [14]:
 labels_list = [ 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
     'SCARED_P_GD', 'WISC_WMI_Sum',
    'WISC_VCI_Sum']

 X = df[df.columns.difference(['IDs']+labels_list)]
 print(X.shape)

(1072, 5511)


In [15]:
Y = df[labels_list]
print(Y.shape)

(1072, 7)


## Split and PCA



In [16]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, shuffle=True)

path = os.path.join(DATA_DIR, 'multilabel_regression_source_level')

# #saving
if not(os.path.exists(path)):
     os.mkdir(path)
np.save(os.path.join(path, 'xtrain.npy'), xtrain)
np.save(os.path.join(path, 'xtest.npy'), xtest)
np.save(os.path.join(path, 'ytrain.npy'), ytrain)
np.save(os.path.join(path, 'ytest.npy'), ytest)


#loading
xtrain = np.load(os.path.join(path, 'xtrain.npy'))
xtest = np.load(os.path.join(path, 'xtest.npy'))
ytrain = np.load(os.path.join(path, 'ytrain.npy'))
ytest = np.load(os.path.join(path, 'ytest.npy'))

In [17]:
age_gender = True
if age_gender:
    train_age_gender = xtrain[:,-2:]
    test_age_gender = xtest[:,-2:]
    xtrain = xtrain[:,:-2]
    xtest = xtest[:,:-2]

In [18]:
# scaling x
norm = preprocessing.MinMaxScaler().fit(xtrain)

# transform training data
xtrain = norm.transform(xtrain)
xtest = norm.transform(xtest)
print(xtrain.shape, xtest.shape)

print('Applying PCA...')
pca = PCA(.90) # 95% variance retained
pca.fit(xtrain)

# transform data
xtrain = pca.transform(xtrain)
xtest = pca.transform(xtest)
print(xtrain.shape, xtest.shape)

(804, 5509) (268, 5509)
Applying PCA...
(804, 220) (268, 220)


In [19]:
if age_gender:
    xtrain = np.concatenate([xtrain, train_age_gender], axis = 1)
    xtest = np.concatenate([xtest, test_age_gender], axis = 1)

In [20]:
# scaling y
min_max_scaler = preprocessing.MinMaxScaler().fit(ytrain)
ytrain = min_max_scaler.transform(ytrain)
ytest = min_max_scaler.transform(ytest)

print(ytrain.shape, ytest.shape)

(804, 7) (268, 7)


In [21]:
ytrain.mean(axis=0)

array([0.4141791 , 0.33175957, 0.62493086, 0.54657267, 0.25214207,
       0.45596238, 0.49326895])

## MODELLING PART

### dummy regressors (to obtain the random baseline):

In [22]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.dummy import DummyRegressor


lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(xtrain, ytrain)
lm_dummy_median = DummyRegressor(strategy = 'median').fit(xtrain, ytrain)
ypred_dummy_mean = lm_dummy_mean.predict(xtest)
ypred_dummy_median = lm_dummy_median.predict(xtest)


print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(ytest,
																	ypred_dummy_mean)))

print("Mean absolute error (dummy): {:.2f}".format(mean_absolute_error(ytest,
																	ypred_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(ytest, ypred_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(ytest, ypred_dummy_median)))


Mean squared error (dummy): 0.04
Mean absolute error (dummy): 0.16
r2_score (dummy mean): -0.01
r2_score (dummy median): -0.03


## SVR regressor

In [23]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Create the SVR regressor
svr = SVR(epsilon=0.2)
#Create the Multioutput Regressor
mor = MultiOutputRegressor(svr)
# Train the regressor
mor = mor.fit(xtrain, ytrain)
# Generate predictions for testing data
ypred = mor.predict(xtest)


print("Mean squared error (SVC): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("Mean abs error (SVC): {:.2f}".format(mean_absolute_error(ytest,
																	ypred)))																	
print("R2 score (SVC): {:.2f}".format(mor.score(xtest, ytest)))


Mean squared error (SVC): 0.04
Mean abs error (SVC): 0.16
R2 score (SVC): -0.02


## random forest reg

In [24]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
#
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=2, random_state=0))
clf.fit(xtrain, ytrain)

# predictions
clf.predict(xtest)

print("Mean squared error (RanFor): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("Mean abs error (RanFor): {:.2f}".format(mean_absolute_error(ytest,
																	ypred)))																		
print("R2 score (RanFor): {:.2f}".format(clf.score(xtest, ytest)))												



Mean squared error (RanFor): 0.04
Mean abs error (RanFor): 0.16
R2 score (RanFor): -0.01


## Linear Regression

In [25]:
# define model
model = LinearRegression()
# fit model
model.fit(xtrain, ytrain)

# Generate predictions for testing data
ypred = model.predict(xtest)

print("Mean squared error (LinReg): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))

print("Mean absolute error (LinReg): {:.2f}".format(mean_absolute_error(ytest,
																	ypred)))																	
print("R2 score (LinReg): {:.2f}".format(model.score(xtest, ytest)))												



Mean squared error (LinReg): 0.05
Mean absolute error (LinReg): 0.17
R2 score (LinReg): -0.16


 ## multi-output meta estimator

In [26]:
max_depth = 20
#330
regr_multirf = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
)
regr_multirf.fit(xtrain, ytrain)
y_multirf = regr_multirf.predict(xtest)


print("Mean squared error (regr_multirf): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (regr_multirf): {:.2f}".format(regr_multirf.score(xtest, ytest)))

Mean squared error (regr_multirf): 0.05
R2 score (regr_multirf): -0.02


## multilayer perceptron with many hidden layers


In [27]:
in_dim = xtrain.shape[1]
out_dim = ytrain.shape[1]


model = Sequential()

model.add(Dense(5, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(4, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(3, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(2, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(1, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))

model.add(Dense(out_dim))
model.compile(loss='mae', optimizer='adam')


model.fit(xtrain, ytrain, epochs=10, batch_size=2, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)

score = tf.keras.metrics.mean_absolute_error(
    ytest, ypred
)
score= np.array(score)
print(score.mean())

score_mse = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)
score_mse =  np.array(score_mse)
print(score_mse.mean())


2021-12-10 23:55:10.833352: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-12-10 23:55:10.864126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 23:55:10.864801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-12-10 23:55:10.865018: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-12-10 23:55:10.866233: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-12-10 23:55:10.867535: I tensorflow/stream_executor/platform/default/d

Epoch 1/10
  1/322 [..............................] - ETA: 0s - loss: 0.4277

2021-12-10 23:55:11.821828: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


322/322 [==============================] - 1s 3ms/step - loss: 0.3248 - val_loss: 0.2279
Epoch 2/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1903 - val_loss: 0.1616
Epoch 3/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1635 - val_loss: 0.1574
Epoch 4/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1624 - val_loss: 0.1574
Epoch 5/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1622 - val_loss: 0.1577
Epoch 6/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1623 - val_loss: 0.1578
Epoch 7/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1622 - val_loss: 0.1577
Epoch 8/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1622 - val_loss: 0.1575
Epoch 9/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1622 - val_loss: 0.1577
Epoch 10/10
322/322 [==============================] - 1s 2ms/step - loss: 0.1622 - val_loss: 0.1578
0.16150719

## mlp 1 hiddenlayer with keras

In [28]:
in_dim = xtrain.shape[1]
out_dim = ytrain.shape[1]


model = Sequential()

model.add(Dense(7, input_dim=in_dim, kernel_initializer='he_uniform', activation='relu'))


model.add(Dense(out_dim))
model.compile(loss='mae', optimizer='adam')


model.fit(xtrain, ytrain, epochs=10, batch_size=2, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)

score = tf.keras.metrics.mean_absolute_error(
    ytest, ypred
)
score= np.array(score)
print(score.mean())

score_mse = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)
score_mse =  np.array(score_mse)
print(score_mse.mean())

Epoch 1/10
322/322 [==============================] - 1s 2ms/step - loss: 1.9378 - val_loss: 0.2497
Epoch 2/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1953 - val_loss: 0.1726
Epoch 3/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1640 - val_loss: 0.1665
Epoch 4/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1623 - val_loss: 0.1665
Epoch 5/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1623 - val_loss: 0.1666
Epoch 6/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1622 - val_loss: 0.1665
Epoch 7/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1623 - val_loss: 0.1665
Epoch 8/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1622 - val_loss: 0.1667
Epoch 9/10
322/322 [==============================] - 0s 2ms/step - loss: 0.1622 - val_loss: 0.1665
Epoch 10/10
322/322 [==============================] - 0s 1ms/step - loss: 0.1623 - val_loss: 0.1666